In [13]:
# RUN THIS BEFORE RUNNING JUPYTER LAB!
# export LD_LIBRARY_PATH=/mnt/lab_data/kundaje/users/chihoim/software/miniforge3/envs/admet_ai/lib/python3.10/site-packages/nvidia/nvjitlink/lib:$LD_LIBRARY_PATH

In [1]:
import pandas as pd
import torch
import numpy as np
from tqdm.notebook import tqdm

In [19]:
import ast

## Genes

In [2]:
llm_embeddings_path = "/mnt/lab_data2/ryanzhao/gene_drug_graph/summary_embeddings_total.tsv"

In [20]:
df_embeds = pd.read_csv(llm_embeddings_path, delimiter='\t', names=["Metadata_Symbol", "ncbi_gene_id", "llm_embedding"])
df_embeds['llm_embedding'] = df_embeds['llm_embedding'].apply(ast.literal_eval)

In [21]:
df_embeds

,Metadata_Symbol,ncbi_gene_id,llm_embedding
0,ASIC1,41,"[-0.0002510891354177147, -0.03559950366616249,..."
1,ACP5,54,"[-0.010736115276813507, 0.007344227284193039, ..."
2,ABAT,18,"[-0.027113983407616615, 0.018105553463101387, ..."
3,ACP1,52,"[-0.00042535888496786356, -0.01791981607675552..."
4,ACADVL,37,"[-0.017211297526955605, -0.03185534104704857, ..."
...,...,...,...
12604,BFP,BFP,"[0.0020580352284014225, 0.014664598740637302, ..."
12605,HcRed,HcRed,"[0.017477931454777718, -0.014081599190831184, ..."
12606,LUCIFERASE,LUCIFERASE,"[-0.009860721416771412, -0.0010026856325566769..."
12607,LacZ,LacZ,"[0.07270344346761703, 0.03857786953449249, -0...."


In [13]:
parquet_path_gene = "/mnt/lab_data/kundaje/users/chihoim/datasets/motive_data/data/st_expanded/orf/target.parquet"

In [28]:
df_tgt_motive = pd.read_parquet(parquet_path_gene, engine='auto')
df_tgt_motive

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_713,X_714,X_715,X_716,X_717,X_718,X_719,X_720,X_721,X_722
Metadata_Symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.223858,0.376254,-0.031659,0.297378,-0.025680,0.297935,-0.185581,0.016823,-0.518060,0.047166,...,0.034199,-0.032264,0.094194,0.231041,-0.117577,0.021722,0.197380,-0.203472,-0.019942,0.124714
A1CF,1.637489,-0.155278,0.137776,-0.163931,-0.328410,0.069691,-0.145191,-0.100056,0.509432,-0.155660,...,-0.098485,0.367990,0.079223,0.205809,-0.013647,0.335760,-0.338771,-0.212275,-0.482585,-0.242562
A2M,0.045647,0.120441,-0.203466,-0.387200,0.211423,-0.575138,-0.110542,-0.240491,0.234466,-0.528015,...,-0.209086,0.253623,0.259652,-0.148212,-0.304498,0.048105,-0.052804,-0.268796,0.293304,-0.039882
A4GALT,-1.256173,-0.218429,-0.481250,-0.172128,0.374500,-0.009168,-0.043310,-0.532803,-0.787149,0.063994,...,0.229561,0.462735,-0.095555,-0.018869,0.236515,0.200817,-0.656448,0.116933,-0.191617,-0.017019
A4GNT,0.382120,0.205143,-0.577680,-0.446890,0.256827,-0.258348,-0.319764,-0.763340,-0.305937,-0.339030,...,-0.310005,0.917883,0.107876,-0.496033,0.698908,0.103408,-0.941137,0.162321,-0.337650,8.692646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWINT,0.434437,0.257166,-0.231369,-0.156888,-0.040366,-0.259545,0.291981,0.167911,-0.387534,0.112846,...,-0.093609,0.021197,-0.588742,0.095454,0.076704,0.073352,-0.200264,-0.039257,-0.063340,-0.348709
ZXDC,-0.088918,0.041629,0.447595,0.787907,-0.539290,-0.256559,-0.249388,-0.490161,0.176942,-0.588061,...,-0.288622,0.057928,0.141798,0.315738,-0.459608,-0.076511,-0.044121,0.007667,0.230921,0.052802
ZYG11B,-1.094900,0.585571,-0.364650,-0.592018,0.154098,-0.074161,-0.195938,-0.206415,-0.000739,0.051372,...,0.066804,-0.315188,0.133892,-0.462622,-0.080032,-0.101599,0.146689,-0.186162,-0.270652,0.181386


In [29]:
df_tgt_motive.drop(df_tgt_motive.columns, axis=1, inplace=True)

In [22]:
llm_embeddings = []
for idx in tqdm(df_tgt_motive.index):
    df_temp = df_embeds[df_embeds["Metadata_Symbol"] == idx]
    llm_embeddings.append(df_temp["llm_embedding"].iloc[0])

  0%|          | 0/11509 [00:00<?, ?it/s]

In [24]:
llm_embeddings = np.array(llm_embeddings)
llm_embeddings.shape

(11509, 1536)

In [31]:
# Step 1: Add q new columns to the DataFrame
num_genes, embed_dim = llm_embeddings.shape

# Step 2: Fill the new columns with values from the embedding matrix
for i in tqdm(range(embed_dim)):
    df_tgt_motive[f'X_{i+1}'] = llm_embeddings[:, i]

  0%|          | 0/1536 [00:00<?, ?it/s]

/tmp/ipykernel_16853/2455661279.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tgt_motive[f'X_{i+1}'] = llm_embeddings[:, i]
/tmp/ipykernel_16853/2455661279.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tgt_motive[f'X_{i+1}'] = llm_embeddings[:, i]
/tmp/ipykernel_16853/2455661279.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-f

In [32]:
df_tgt_motive

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_1527,X_1528,X_1529,X_1530,X_1531,X_1532,X_1533,X_1534,X_1535,X_1536
Metadata_Symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.022618,-0.019301,0.003651,0.034100,-0.071985,-0.018518,0.070897,0.008416,-0.031991,-0.018507,...,-0.010113,0.016822,0.023118,-0.015756,-0.029011,0.029860,0.015310,0.018018,0.003213,0.017768
A1CF,0.007736,0.034429,0.020594,0.062594,-0.046251,-0.014624,0.074275,0.014294,-0.023055,-0.019428,...,-0.008431,-0.003462,0.016496,0.000022,-0.041635,0.000356,-0.005861,0.011863,0.002640,0.019617
A2M,0.010336,-0.000821,0.023685,0.037541,-0.074047,0.036731,0.045346,-0.000997,-0.011764,-0.009065,...,-0.018399,-0.026182,0.013271,-0.019423,-0.013485,0.012911,-0.008171,0.012281,-0.019951,0.012855
A4GALT,-0.018249,0.001610,0.025440,0.034078,-0.057302,-0.002469,0.021177,-0.015818,-0.011188,-0.024332,...,0.000981,-0.002381,0.021958,-0.001541,-0.050699,0.021471,-0.017458,0.054770,0.018509,0.006134
A4GNT,0.000131,0.027910,0.033866,0.011508,-0.053419,-0.023847,0.017118,0.008899,-0.025948,-0.030080,...,0.008911,-0.003105,0.016310,-0.015155,-0.048202,0.005079,-0.005555,0.037329,-0.000628,0.025047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWINT,-0.011417,0.058268,0.013015,0.036128,-0.056833,0.004192,0.052849,-0.045114,-0.046944,-0.034970,...,0.022418,0.005822,0.015493,-0.009611,-0.004227,-0.013977,-0.001874,0.020218,0.036823,0.019083
ZXDC,0.031150,0.014676,0.016094,0.061932,0.029744,0.037214,0.042033,0.012935,-0.010791,-0.034632,...,-0.002505,0.026839,0.023138,0.010681,0.001314,-0.024210,-0.012520,0.006174,0.003162,0.021501
ZYG11B,0.001350,0.037213,0.031999,0.051236,-0.039641,0.029523,0.041736,0.023939,-0.005482,-0.029999,...,0.029332,0.030189,0.020487,0.026047,-0.037094,-0.012333,0.013999,0.027261,-0.039594,-0.005622


In [33]:
df_tgt_motive.to_parquet("/mnt/lab_data/kundaje/users/chihoim/datasets/motive_data/data/st_expanded/orf/target_llm.parquet")

## Drugs

In [2]:
parquet_path_drug = "/mnt/lab_data/kundaje/users/chihoim/datasets/motive/data/st_expanded/orf/source.parquet"

In [3]:
df_src_motive = pd.read_parquet(parquet_path_drug, engine='auto')
df_src_motive

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_728,X_729,X_730,X_731,X_732,X_733,X_734,X_735,X_736,X_737
Metadata_InChIKey,,,,,,,,,,,,,,,,,,,,,
AAAQFGUYHFJNHI,0.609696,-0.628469,0.186290,-0.051378,0.062075,0.106780,-0.700471,1.097518,-0.609469,0.013312,...,0.279681,0.766521,0.850457,0.400008,-0.707736,-0.440621,-0.016084,-0.727890,0.915430,-0.117364
AAFJXZWCNVJTMK,0.954355,1.905620,-1.620347,0.602247,-2.034726,-1.281924,-1.446034,-1.694948,1.856530,1.366390,...,1.165352,-0.107811,-2.983105,-2.387725,2.486181,0.859030,-0.389957,2.230050,0.932263,1.715021
AAKJLRGGTJKAMG,-0.037173,-0.637096,-0.054307,-0.811288,0.437335,0.562220,-0.081636,0.715513,-0.148607,-0.460636,...,0.298911,0.371913,0.238583,0.267379,0.339304,-0.254494,0.055256,-0.010962,1.176500,0.468096
AAQOQKQBGPPFNS,-0.444124,0.817667,-0.557278,-0.920106,-0.442352,-1.073516,-0.577252,-0.253010,0.031081,0.181129,...,-0.913836,-0.847261,-0.150615,0.565554,-0.135296,-1.038276,0.073130,-0.717930,-0.516670,-0.579128
ABACVOXFUHDKNZ,0.671451,-0.894071,0.840540,0.623787,0.317617,0.406311,0.163020,-0.257224,-0.052855,-0.618134,...,-0.595474,-0.299790,-0.256531,-0.012293,-0.548202,-0.347855,0.161040,-0.272391,-0.340551,-0.279669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZHLYYDVIOPZBE,1.181892,-0.449215,-0.027586,0.323213,-0.138903,-0.371968,-0.056930,0.162713,0.124127,0.618876,...,0.510542,0.568454,0.119904,0.704001,0.301344,0.066366,-0.145060,0.189414,0.478701,0.382980
ZZORFUFYDOWNEF,-0.158592,-0.340769,0.226615,-0.360913,-0.071489,-0.080857,0.185982,0.069592,-0.267350,-0.125759,...,0.114448,0.141367,0.369393,0.306468,0.567638,0.256222,0.832597,0.805979,0.291511,0.580806
ZZQNEJILGNNOEP,0.951287,-0.293756,0.024565,-0.632311,0.293068,-0.430618,0.310687,0.247387,-0.353484,0.469128,...,0.691066,0.940162,-0.214920,-0.260958,0.286002,0.588993,0.260245,0.279461,0.937752,-0.232397


In [4]:
df_src_all = pd.read_csv("/mnt/lab_data/kundaje/users/chihoim/datasets/motive/inputs/compound/meta.csv.gz", compression='gzip')
df_src_all['Metadata_InChIKey_0'] = df_src_all['Metadata_InChIKey'].apply(lambda x: str(x).split('-')[0])
df_src_all

,Metadata_JCP2022,Metadata_InChIKey,Metadata_InChI,Metadata_InChIKey_0
0,JCP2022_000001,AAAHWCWPZPSPIW-UHFFFAOYSA-N,InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...,AAAHWCWPZPSPIW
1,JCP2022_000002,AAAJHRMBUHXWLD-UHFFFAOYSA-N,InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...,AAAJHRMBUHXWLD
2,JCP2022_000003,AAALVYBICLMAMA-UHFFFAOYSA-N,InChI=1S/C20H15N3O2/c24-19-15-11-17(21-13-7-3-...,AAALVYBICLMAMA
3,JCP2022_000004,AAANUZMCJQUYNX-UHFFFAOYSA-N,InChI=1S/C13H22N4O2S/c1-2-7-16-13(5-6-15-16)20...,AAANUZMCJQUYNX
4,JCP2022_000005,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...,AAAQFGUYHFJNHI
...,...,...,...,...
116748,JCP2022_116750,ZZZTXDPKNAOZPM-UHFFFAOYSA-N,InChI=1S/C22H27N3O2/c1-14-18(22(27)24-15(2)23-...,ZZZTXDPKNAOZPM
116749,JCP2022_116751,ZZZUOLMMTJKOGE-UHFFFAOYSA-N,InChI=1S/C19H17N5O2/c1-23-9-6-13-7-10-24(19(26...,ZZZUOLMMTJKOGE
116750,JCP2022_116752,ZZZZIBSVRUABIA-UHFFFAOYSA-N,InChI=1S/C19H13F6N3O2S/c1-11-26-6-5-17(27-11)1...,ZZZZIBSVRUABIA
116751,JCP2022_116753,ZZZZVNLQWWXZTQ-UHFFFAOYSA-N,InChI=1S/C14H20N2O3/c1-18-13-5-3-2-4-12(13)14(...,ZZZZVNLQWWXZTQ


In [5]:
metadata_inchis = []
metadata_inchikeys = []
for idx in df_src_motive.index:
    df_temp = df_src_all[df_src_all["Metadata_InChIKey_0"] == idx]
    metadata_inchis.append(df_temp["Metadata_InChI"].iloc[0])
    metadata_inchikeys.append(df_temp["Metadata_InChIKey"].iloc[0])

In [6]:
df_src_motive["Metadata_InChI"] = metadata_inchis
df_src_motive["Metadata_InChIKey"] = metadata_inchikeys
df_src_new = df_src_motive[["Metadata_InChIKey", "Metadata_InChI"]]
df_src_new

,Metadata_InChIKey,Metadata_InChI
Metadata_InChIKey,,
AAAQFGUYHFJNHI,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...
AAFJXZWCNVJTMK,AAFJXZWCNVJTMK-UHFFFAOYSA-N,InChI=1S/C6H10O4/c7-5(3-1-9-3)6(8)4-2-10-4/h3-...
AAKJLRGGTJKAMG,AAKJLRGGTJKAMG-UHFFFAOYSA-N,InChI=1S/C22H23N3O4/c1-4-16-6-5-7-17(12-16)25-...
AAQOQKQBGPPFNS,AAQOQKQBGPPFNS-UHFFFAOYSA-N,"InChI=1S/C11H17BrN/c1-4-13(2,3)9-10-7-5-6-8-11..."
ABACVOXFUHDKNZ,ABACVOXFUHDKNZ-UHFFFAOYSA-N,InChI=1S/C20H19ClN2O3S/c1-3-20(24)18-12-19(14-...
...,...,...
ZZHLYYDVIOPZBE,ZZHLYYDVIOPZBE-UHFFFAOYSA-N,InChI=1S/C18H22N2S/c1-14(12-19(2)3)13-20-15-8-...
ZZORFUFYDOWNEF,ZZORFUFYDOWNEF-UHFFFAOYSA-N,InChI=1S/C12H14N4O4S/c1-19-11-7-10(14-12(15-11...
ZZQNEJILGNNOEP,ZZQNEJILGNNOEP-UHFFFAOYSA-N,InChI=1S/C24H25FN4O2/c1-15-4-3-10-29-23(15)26-...


In [ ]:
# For testing:
df_src_new = df_src_new.head(10)
df_src_new

### Converting molecule InChI's into SMILES

In [7]:
from rdkit import Chem
import requests

In [8]:
def inchi_to_mol(inchi):
    try:
        # Direct conversion from InChI to SMILES via RDKit
        mol = Chem.MolFromInchi(inchi)
        if mol:
            return mol
        return None
    except:
        return None

In [9]:
def inchi_to_smiles(inchi):
    try:
        # Direct conversion from InChI to SMILES via RDKit
        mol = Chem.MolFromInchi(inchi)
        if mol:
            smiles = Chem.MolToSmiles(mol)
            return smiles
        return None
    except:
        return None

In [10]:
# df_src_new["SMILES"] = df_src_new["Metadata_InChI"].apply(inchi_to_smiles)

for idx in df_src_new.index:
    smiles = inchi_to_smiles(df_src_new.at[idx, "Metadata_InChI"])
    if not smiles:
        inc_key = df_src_new.at[idx, "Metadata_InChIKey"]
        r = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{inc_key}/property/CanonicalSMILES/JSON').json()
        smiles = r['PropertyTable']['Properties'][0]['CanonicalSMILES']
        print("Filled SMILES in manually", smiles)
    df_src_new.at[idx, "SMILES"] = smiles

    # m = inchi_to_mol(df_src_new.at[idx, "Metadata_InChI"])
    # df_src_new.at[idx, "Mol"] = m

df_src_new

/tmp/ipykernel_191420/1463697558.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_src_new.at[idx, "SMILES"] = smiles
[23:23:30] Explicit valence for atom # 10 N, 4, is greater than permitted
[23:23:30] ERROR: Explicit valence for atom # 10 N, 4, is greater than permitted



Filled SMILES in manually C1=CC=C2C(=C1)[N+](=C(N=[N+]2[O-])N)[O-]


,Metadata_InChIKey,Metadata_InChI,SMILES
Metadata_InChIKey,,,
AAAQFGUYHFJNHI,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...,CCN=C(O)CC1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-n2c(C...
AAFJXZWCNVJTMK,AAFJXZWCNVJTMK-UHFFFAOYSA-N,InChI=1S/C6H10O4/c7-5(3-1-9-3)6(8)4-2-10-4/h3-...,OC(C1CO1)C(O)C1CO1
AAKJLRGGTJKAMG,AAKJLRGGTJKAMG-UHFFFAOYSA-N,InChI=1S/C22H23N3O4/c1-4-16-6-5-7-17(12-16)25-...,C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1
AAQOQKQBGPPFNS,AAQOQKQBGPPFNS-UHFFFAOYSA-N,"InChI=1S/C11H17BrN/c1-4-13(2,3)9-10-7-5-6-8-11...",CC[N+](C)(C)Cc1ccccc1Br
ABACVOXFUHDKNZ,ABACVOXFUHDKNZ-UHFFFAOYSA-N,InChI=1S/C20H19ClN2O3S/c1-3-20(24)18-12-19(14-...,CCC(=O)c1cc(-c2ccc(Cl)cc2)n(-c2ccc(S(N)(=O)=O)...
...,...,...,...
ZZHLYYDVIOPZBE,ZZHLYYDVIOPZBE-UHFFFAOYSA-N,InChI=1S/C18H22N2S/c1-14(12-19(2)3)13-20-15-8-...,CC(CN(C)C)CN1c2ccccc2Sc2ccccc21
ZZORFUFYDOWNEF,ZZORFUFYDOWNEF-UHFFFAOYSA-N,InChI=1S/C12H14N4O4S/c1-19-11-7-10(14-12(15-11...,COc1cc(NS(=O)(=O)c2ccc(N)cc2)nc(OC)n1
ZZQNEJILGNNOEP,ZZQNEJILGNNOEP-UHFFFAOYSA-N,InChI=1S/C24H25FN4O2/c1-15-4-3-10-29-23(15)26-...,Cc1nc2c(C)cccn2c(=O)c1CCN1CCC(c2noc3cc(F)ccc23...


In [11]:
df_src_new.isna().any().any()

False

### Converting molecule SMILES into chemprop embeddings

In [12]:
from chemprop.utils import load_checkpoint
from chemprop.features import get_features_generator

In [13]:
model_path = "/mnt/lab_data/kundaje/users/chihoim/cs224w/chemprop_models/kyle_admet_model.pt"

In [14]:
# load pre-trained model
admet_model = load_checkpoint(model_path, device=torch.device('cuda:3'))
admet_model.eval()
# admet_model

/mnt/lab_data/kundaje/users/chihoim/software/miniforge3/envs/admet_ai/lib/python3.10/site-packages/chemprop/utils.py:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda


MoleculeModel(
  (sigmoid): Sigmoid()
  (encoder): MPN(
    (encoder): ModuleList(
      (0): MPNEncoder(
        (dropout): Dropout(p=0.0, inplace=False)
        (act_func): ReLU()
        (W_i): Linear(in_features=147, out_features=300, bias=False)
        (W_h): Linear(in_features=300, out_features=300, bias=False)
        (W_o): Linear(in_features=433, out_features=300, bias=True)
      )
    )
  )
  (readout): Sequential(
    (0): Dropout(p=0.0, inplace=False)
    (1): Linear(in_features=300, out_features=300, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.0, inplace=False)
    (4): Linear(in_features=300, out_features=31, bias=True)
  )
)

In [15]:
feature_gen = get_features_generator("rdkit_2d_normalized")

In [16]:
fingerprints = []
for smi in tqdm(df_src_new["SMILES"].tolist()):
    fingerprints.append(feature_gen(smi))

  0%|          | 0/3632 [00:00<?, ?it/s]

[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerator
[22:01:31] DEPRECATION WARNING: please use MorganGenerat

3632


[22:02:57] DEPRECATION WARNING: please use MorganGenerator
[22:02:57] DEPRECATION WARNING: please use MorganGenerator
[22:02:57] DEPRECATION WARNING: please use MorganGenerator
[22:02:57] DEPRECATION WARNING: please use MorganGenerator
[22:02:57] DEPRECATION WARNING: please use MorganGenerator
[22:02:57] DEPRECATION WARNING: please use MorganGenerator
[22:02:57] DEPRECATION WARNING: please use MorganGenerator
[22:02:57] DEPRECATION WARNING: please use MorganGenerator
[22:02:57] DEPRECATION WARNING: please use MorganGenerator


In [24]:
# for ADMET-AI model with RDKit features

embeddings = []
with torch.no_grad():
    for smi, fp in tqdm(zip(df_src_new["SMILES"].tolist(), fingerprints), total=len(fingerprints)):
        # embedding = admet_model.encoder(batch=[[smi]], features_batch=[fp])
        embedding = admet_model.encoder(batch=[[smi]]) # for no RDKit feature model
        embeddings.append(embedding)

  0%|          | 0/3632 [00:00<?, ?it/s]

In [17]:
# for ADMET-AI model with NO RDKit features

embeddings = []
with torch.no_grad():
    for smi in tqdm(df_src_new["SMILES"].tolist()):
        embedding = admet_model.encoder(batch=[[smi]]) # for no RDKit feature model
        embeddings.append(embedding)

  0%|          | 0/3632 [00:00<?, ?it/s]

In [18]:
drug_embeds = torch.cat(embeddings, dim=0).detach().cpu().numpy()
drug_embeds.shape

(3632, 300)

In [19]:
# Check which rows contain NaN
rows_with_nan = np.any(np.isnan(drug_embeds), axis=1)

# Get the indices of the rows with NaN values
indices_with_nan = np.where(rows_with_nan)[0]

print(indices_with_nan)

[]


In [20]:
print(df_src_new["SMILES"].iloc[561])

O=c1c2ccccc2[se]n1-c1ccccc1


In [21]:
# Check if any element is NaN
if np.any(np.isnan(drug_embeds)):
    print("Array contains NaN")
else:
    print("Array does not contain NaN")

Array does not contain NaN


### Put the ADMET-AI embeddings into the corresponding drug parquet file

In [22]:
# Step 1: Add q new columns to the DataFrame
num_drugs, embed_dim = drug_embeds.shape

# for i in range(embed_dim):
#     df_src_new[f'X_{i+1}'] = np.nan  # Initialize with NaN or zeros

# Step 2: Fill the new columns with values from the embedding matrix
for i in range(embed_dim):
    df_src_new[f'X_{i+1}'] = drug_embeds[:, i]

/tmp/ipykernel_191420/3741102308.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_src_new[f'X_{i+1}'] = drug_embeds[:, i]
/tmp/ipykernel_191420/3741102308.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_src_new[f'X_{i+1}'] = drug_embeds[:, i]
/tmp/ipykernel_191420/3741102308.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

,Metadata_InChIKey,Metadata_InChI,SMILES,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_291,X_292,X_293,X_294,X_295,X_296,X_297,X_298,X_299,X_300
Metadata_InChIKey,,,,,,,,,,,,,,,,,,,,,
AAAQFGUYHFJNHI,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...,CCN=C(O)CC1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-n2c(C...,0.086071,0.324422,0.294836,0.149264,0.114811,0.095877,0.216355,...,0.374873,0.140769,0.057679,0.218686,0.385161,0.162404,0.275703,0.243085,0.224357,0.374692
AAFJXZWCNVJTMK,AAFJXZWCNVJTMK-UHFFFAOYSA-N,InChI=1S/C6H10O4/c7-5(3-1-9-3)6(8)4-2-10-4/h3-...,OC(C1CO1)C(O)C1CO1,0.100499,0.250443,0.029770,0.541155,0.099237,0.029933,0.000000,...,1.055983,0.000000,0.950427,0.000000,0.000000,0.000000,0.442132,1.609445,1.344254,0.182736
AAKJLRGGTJKAMG,AAKJLRGGTJKAMG-UHFFFAOYSA-N,InChI=1S/C22H23N3O4/c1-4-16-6-5-7-17(12-16)25-...,C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1,0.064144,0.271283,0.100916,0.084390,0.229819,0.060127,0.125173,...,0.098482,0.233484,0.040919,0.118526,0.096298,0.585645,0.331582,0.069432,0.170536,0.514175
AAQOQKQBGPPFNS,AAQOQKQBGPPFNS-UHFFFAOYSA-N,"InChI=1S/C11H17BrN/c1-4-13(2,3)9-10-7-5-6-8-11...",CC[N+](C)(C)Cc1ccccc1Br,0.311090,0.965525,0.026334,0.132432,0.167499,0.000000,0.122521,...,0.611607,0.395680,0.130848,0.000000,0.108927,0.029612,0.347373,0.458842,0.025851,0.210130
ABACVOXFUHDKNZ,ABACVOXFUHDKNZ-UHFFFAOYSA-N,InChI=1S/C20H19ClN2O3S/c1-3-20(24)18-12-19(14-...,CCC(=O)c1cc(-c2ccc(Cl)cc2)n(-c2ccc(S(N)(=O)=O)...,0.093253,0.291315,0.537051,0.189960,0.105425,0.528248,0.000000,...,0.283222,0.476538,0.004073,0.135753,0.272757,0.040463,0.056846,0.078076,0.211507,0.802861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZHLYYDVIOPZBE,ZZHLYYDVIOPZBE-UHFFFAOYSA-N,InChI=1S/C18H22N2S/c1-14(12-19(2)3)13-20-15-8-...,CC(CN(C)C)CN1c2ccccc2Sc2ccccc21,0.135866,0.379057,0.129069,0.022840,0.182013,0.018845,0.000000,...,0.378078,0.288000,0.008788,0.006660,0.000000,0.250174,0.415128,0.196468,0.131180,0.301105
ZZORFUFYDOWNEF,ZZORFUFYDOWNEF-UHFFFAOYSA-N,InChI=1S/C12H14N4O4S/c1-19-11-7-10(14-12(15-11...,COc1cc(NS(=O)(=O)c2ccc(N)cc2)nc(OC)n1,0.315861,0.396481,0.556970,0.392000,0.063804,0.532001,0.425790,...,0.815922,0.106350,0.073263,0.350915,0.399671,0.780373,0.413587,0.040447,0.101905,0.820958
ZZQNEJILGNNOEP,ZZQNEJILGNNOEP-UHFFFAOYSA-N,InChI=1S/C24H25FN4O2/c1-15-4-3-10-29-23(15)26-...,Cc1nc2c(C)cccn2c(=O)c1CCN1CCC(c2noc3cc(F)ccc23...,0.082220,0.401725,0.192793,0.018812,0.026888,0.193564,0.106823,...,0.289704,0.085281,0.146567,0.221666,0.137776,0.170828,0.266679,0.045923,0.178218,0.305044


In [25]:
df_src_new

,Metadata_InChIKey,Metadata_InChI,SMILES,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_291,X_292,X_293,X_294,X_295,X_296,X_297,X_298,X_299,X_300
Metadata_InChIKey,,,,,,,,,,,,,,,,,,,,,
AAAQFGUYHFJNHI,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...,CCN=C(O)CC1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-n2c(C...,0.086071,0.324422,0.294836,0.149264,0.114811,0.095877,0.216355,...,0.374873,0.140769,0.057679,0.218686,0.385161,0.162404,0.275703,0.243085,0.224357,0.374692
AAFJXZWCNVJTMK,AAFJXZWCNVJTMK-UHFFFAOYSA-N,InChI=1S/C6H10O4/c7-5(3-1-9-3)6(8)4-2-10-4/h3-...,OC(C1CO1)C(O)C1CO1,0.100499,0.250443,0.029770,0.541155,0.099237,0.029933,0.000000,...,1.055983,0.000000,0.950427,0.000000,0.000000,0.000000,0.442132,1.609445,1.344254,0.182736
AAKJLRGGTJKAMG,AAKJLRGGTJKAMG-UHFFFAOYSA-N,InChI=1S/C22H23N3O4/c1-4-16-6-5-7-17(12-16)25-...,C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1,0.064144,0.271283,0.100916,0.084390,0.229819,0.060127,0.125173,...,0.098482,0.233484,0.040919,0.118526,0.096298,0.585645,0.331582,0.069432,0.170536,0.514175
AAQOQKQBGPPFNS,AAQOQKQBGPPFNS-UHFFFAOYSA-N,"InChI=1S/C11H17BrN/c1-4-13(2,3)9-10-7-5-6-8-11...",CC[N+](C)(C)Cc1ccccc1Br,0.311090,0.965525,0.026334,0.132432,0.167499,0.000000,0.122521,...,0.611607,0.395680,0.130848,0.000000,0.108927,0.029612,0.347373,0.458842,0.025851,0.210130
ABACVOXFUHDKNZ,ABACVOXFUHDKNZ-UHFFFAOYSA-N,InChI=1S/C20H19ClN2O3S/c1-3-20(24)18-12-19(14-...,CCC(=O)c1cc(-c2ccc(Cl)cc2)n(-c2ccc(S(N)(=O)=O)...,0.093253,0.291315,0.537051,0.189960,0.105425,0.528248,0.000000,...,0.283222,0.476538,0.004073,0.135753,0.272757,0.040463,0.056846,0.078076,0.211507,0.802861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZHLYYDVIOPZBE,ZZHLYYDVIOPZBE-UHFFFAOYSA-N,InChI=1S/C18H22N2S/c1-14(12-19(2)3)13-20-15-8-...,CC(CN(C)C)CN1c2ccccc2Sc2ccccc21,0.135866,0.379057,0.129069,0.022840,0.182013,0.018845,0.000000,...,0.378078,0.288000,0.008788,0.006660,0.000000,0.250174,0.415128,0.196468,0.131180,0.301105
ZZORFUFYDOWNEF,ZZORFUFYDOWNEF-UHFFFAOYSA-N,InChI=1S/C12H14N4O4S/c1-19-11-7-10(14-12(15-11...,COc1cc(NS(=O)(=O)c2ccc(N)cc2)nc(OC)n1,0.315861,0.396481,0.556970,0.392000,0.063804,0.532001,0.425790,...,0.815922,0.106350,0.073263,0.350915,0.399671,0.780373,0.413587,0.040447,0.101905,0.820958
ZZQNEJILGNNOEP,ZZQNEJILGNNOEP-UHFFFAOYSA-N,InChI=1S/C24H25FN4O2/c1-15-4-3-10-29-23(15)26-...,Cc1nc2c(C)cccn2c(=O)c1CCN1CCC(c2noc3cc(F)ccc23...,0.082220,0.401725,0.192793,0.018812,0.026888,0.193564,0.106823,...,0.289704,0.085281,0.146567,0.221666,0.137776,0.170828,0.266679,0.045923,0.178218,0.305044


In [23]:
df_src_new.isna().any().any()

False

In [26]:
df_save = df_src_new.drop(columns=["Metadata_InChIKey", "Metadata_InChI", "SMILES"])
df_save

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_291,X_292,X_293,X_294,X_295,X_296,X_297,X_298,X_299,X_300
Metadata_InChIKey,,,,,,,,,,,,,,,,,,,,,
AAAQFGUYHFJNHI,0.086071,0.324422,0.294836,0.149264,0.114811,0.095877,0.216355,0.259290,0.131691,0.346182,...,0.374873,0.140769,0.057679,0.218686,0.385161,0.162404,0.275703,0.243085,0.224357,0.374692
AAFJXZWCNVJTMK,0.100499,0.250443,0.029770,0.541155,0.099237,0.029933,0.000000,0.000000,0.607817,0.732442,...,1.055983,0.000000,0.950427,0.000000,0.000000,0.000000,0.442132,1.609445,1.344254,0.182736
AAKJLRGGTJKAMG,0.064144,0.271283,0.100916,0.084390,0.229819,0.060127,0.125173,0.555511,0.328331,0.115810,...,0.098482,0.233484,0.040919,0.118526,0.096298,0.585645,0.331582,0.069432,0.170536,0.514175
AAQOQKQBGPPFNS,0.311090,0.965525,0.026334,0.132432,0.167499,0.000000,0.122521,0.000000,0.188787,0.252621,...,0.611607,0.395680,0.130848,0.000000,0.108927,0.029612,0.347373,0.458842,0.025851,0.210130
ABACVOXFUHDKNZ,0.093253,0.291315,0.537051,0.189960,0.105425,0.528248,0.000000,0.037857,0.160742,0.070305,...,0.283222,0.476538,0.004073,0.135753,0.272757,0.040463,0.056846,0.078076,0.211507,0.802861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZHLYYDVIOPZBE,0.135866,0.379057,0.129069,0.022840,0.182013,0.018845,0.000000,0.039056,0.344003,0.213761,...,0.378078,0.288000,0.008788,0.006660,0.000000,0.250174,0.415128,0.196468,0.131180,0.301105
ZZORFUFYDOWNEF,0.315861,0.396481,0.556970,0.392000,0.063804,0.532001,0.425790,0.394367,0.094132,0.340305,...,0.815922,0.106350,0.073263,0.350915,0.399671,0.780373,0.413587,0.040447,0.101905,0.820958
ZZQNEJILGNNOEP,0.082220,0.401725,0.192793,0.018812,0.026888,0.193564,0.106823,0.403711,0.066419,0.121039,...,0.289704,0.085281,0.146567,0.221666,0.137776,0.170828,0.266679,0.045923,0.178218,0.305044


In [28]:
df_save.to_parquet("/mnt/lab_data/kundaje/users/chihoim/datasets/motive_data/data/st_expanded/orf/source_admet.parquet")